In [1]:
import os,sys
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PeriodicTable
from rdkit.Chem import rdPartialCharges
from rdkit.Chem import Descriptors
import pandas as pd

df = pd.read_excel('generated_molecules_layers8_units[16, 16].xlsx') ####LOCATION OF THE EXCEL FILE
pse = Chem.GetPeriodicTable()


In [2]:
home_dir = os.getcwd()
print(df)

                                     Original_SMILES        mu     alpha  \
0  [H]C1([H])O[C@]12C([H])([H])[C@]1([H])C([H])([...  0.199986  0.375006   
1        [H]C#CC([H])([H])[C@]1([H])OC([H])([H])C1=O -0.816216 -1.402574   
2  [H]ONC1C([H])([H])C([H])([H])C([H])([H])[C@]12... -0.632855  0.668573   

       homo      lumo       gap        r2      zpve        cv        u0  ...  \
0  0.282529 -0.915383 -1.055220 -0.035543 -0.536086  0.044669  0.321575  ...   
1 -0.000204  0.791031  0.789536 -1.503268 -0.619537 -1.682616  0.344594  ...   
2  0.549333  1.577222  1.298640 -0.477336  1.050343 -0.128274  0.264766  ...   

             Generated_SMILES_1             Generated_SMILES_2  \
0       [C@@H1]/[C@@]C[NH1+1]\O                           C[H]   
1                        O=C[H]  NS[C@@][NH1]N(F)O[NH1+1][C-1]   
2  NC/C\[C@@][C-1]C(/OCF)/[N+1]                  [H][C@@]=N[H]   

    Generated_SMILES_3 Generated_SMILES_4 Generated_SMILES_5  \
0     CC(C)N\[C@@]ON=C           [NH1][H]    

In [3]:
og_dipole = df['mu']
og_zpe = df['zpve']
predicted_dipole = np.zeros((3,5), dtype='float32')
dipole_mae = 0
zpe_mae = 0
predicted_zpe = np.zeros((3,5), dtype='float32')
count_dipole = 0
count_zpe = 0

for i in range(1,6):
    if os.path.exists(home_dir + '/Generated_SMILES_' + str(i) + '/thermodynamic_properties'):
        file = open(home_dir + '/Generated_SMILES_' + str(i) + '/thermodynamic_properties', 'r')
        lines = file.readlines()
        for j, line in enumerate(lines[1:]):
            splits = line.split()
            if float(splits[-2]) == 0:
                predicted_dipole[j][i-1] = og_dipole[j]
            else:
                predicted_dipole[j][i-1] = float(splits[-2]) 
                count_dipole = count_dipole +1
                dipole_mae = dipole_mae + np.abs((predicted_dipole[j][i-1] - np.abs(og_dipole[j])))
            if float(splits[-1]) == 0:
                predicted_zpe[j][i-1] = og_zpe[j]
            else:
                predicted_zpe[j][i-1] = float(splits[-1])
                count_zpe = count_zpe + 1
                zpe_mae = zpe_mae + np.abs((predicted_zpe[j][i-1] - np.abs(og_zpe[j])))
        file.close()
print(predicted_zpe)
dipole_mae = dipole_mae/count_dipole
print(dipole_mae)
zpe_mae = zpe_mae/count_zpe 
print(zpe_mae)
file = open('errors', 'w')
file.write('Dipole ' + str(dipole_mae) + '\n')
file.write('ZPE ' + str(zpe_mae) + '\n')
file.close()

[[-0.53608596 -0.53608596  0.15566656  0.0195607   0.06033339]
 [ 0.02897266 -0.6195374   0.01786149  0.06935352  0.009795  ]
 [ 1.0503432   0.02695475  1.0503432   0.04870112  0.12066107]]
0.5762490947212896
0.6679576735433719
